COLAB

In [2]:
folder_path = '/content/drive/MyDrive/Data/colabs_data/MOA_kaggle/'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!cp '/content/drive/MyDrive/Data/colabs_data/MOA_kaggle/quanvh8_funcs.py' .

KAGGLE

In [ ]:
# folder_path = '../input/lish-moa/'

In [ ]:
# !cp '../input/coded-file/quanvh8_funcs.py' .

In [8]:
'''ENSEMBLE NETS
Inspire by https://www.kaggle.com/c/otto-group-product-classification-challenge/discussion/14335'''

import numpy as np, pandas as pd, copy, tensorflow as tf, matplotlib.pyplot as plt, sklearn

from tensorflow import feature_column as fc
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import (Dense, DenseFeatures, Dropout, 
                                     BatchNormalization, Embedding, Input, Concatenate, Average,
                                     InputLayer, Lambda)
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras import backend as K, Sequential, Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop

import tensorflow_addons as tfa
from tensorflow_addons.layers import WeightNormalization

from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans

from quanvh8_funcs import (DerivedFeatures, kfolds_bagging_training, voting_predict,
                           kolds_stacked_ensemble_training, stacked_ensemble_predict )

import sys

def log_loss_metric(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()
    return bce(y_true, y_pred).numpy()

print(pd.__version__)
print(tf.__version__)

1.1.4
2.3.0


In [9]:
# Loading data and encoding


raw_test = pd.read_csv(folder_path + 'test_features.csv')
raw_train = pd.read_csv(folder_path + 'train_features.csv')
raw_targets = pd.read_csv(folder_path + 'train_targets_scored.csv')

# Phân loại dữ liệu
cols_id = ['sig_id']
cols_to_remove = ['cp_type']
cols_fts = [i for i in raw_train.columns if i not in cols_id +cols_to_remove]
cols_gene = [col for col in raw_train.columns if col.startswith("g-")]
cols_cell = [col for col in raw_train.columns if col.startswith("c-")]
cols_experiment = [col for col in cols_fts if col not in cols_gene+cols_cell]
cols_target = [i for i in raw_targets.columns if i not in cols_id]
num_fts, num_labels = len(cols_fts), len(cols_target)

# xử lý categorical
def transform_data(input_data):
    '''Clean data and encoding
        * input_data: table '''
    out = input_data.copy()
    out['cp_dose'] = out['cp_dose'].map({'D1':0, 'D2':1})
    out['cp_time'] = out['cp_time']/72
    
    return out

to_train = transform_data(raw_train[raw_train['cp_type'] != 'ctl_vehicle'])
to_train_targets = raw_targets.iloc[to_train.index]
full_pred  = transform_data(raw_test)
to_pred = full_pred[full_pred['cp_type'] != 'ctl_vehicle']

In [10]:
# preprocessing pipeline
def pipe_line_builder(quantiles_num, pca_dims, kmean_clusters):
    '''Dựng pipe line cho từng nhóm columns
    :quantiles_num: int: số quantile khi normalise
    :pca_dims: int: số chiều pca'''
    norm = QuantileTransformer(n_quantiles=quantiles_num,random_state=0, output_distribution="normal")
    pca = PCA(n_components = pca_dims)
    derived_ft = DerivedFeatures(n_clusters = kmean_clusters)
    tsne = sklearn.manifold.TSNE(n_components = int(pca_dims/2))
    isomap = sklearn.manifold.Isomap(n_neighbors = 128, n_components = int(pca_dims/2) )

    p_derived_ft = Pipeline([
        ('norm', norm), 
        ('derived', derived_ft)])
    
    p_isomap = Pipeline([
        ('norm', norm), 
        ('isomap', isomap)])

    p_norm_pca = Pipeline([ 
        ('norm', norm),
        ('pca', pca) ])
    return FeatureUnion([
        ('norm', norm), 
        ('norm_pca', p_norm_pca),
        ('derived', p_derived_ft),
#         ('isomap', p_isomap)
    ])

# Dựng pipe transform data

pipe = Pipeline([
    ('norm_pca', ColumnTransformer([
                     ('gene', pipe_line_builder(quantiles_num = 200, pca_dims = 600, kmean_clusters = 5), cols_gene),
                     ('cell', pipe_line_builder(quantiles_num = 200, pca_dims = 50, kmean_clusters = 5), cols_cell),
                    ]) 
    ), 
    ('var', VarianceThreshold(0.01)) 
])

pipe = ColumnTransformer([
    ('gene_cell', pipe, cols_gene+ cols_cell),
    ('experiment', 'passthrough', cols_experiment)
])

In [11]:
# Transform data
pipe.fit(to_train[cols_fts].append(to_pred[cols_fts]))
X_train = pipe.transform(to_train[cols_fts])
X_pred = pipe.transform(to_pred[cols_fts])
y_train = to_train_targets[cols_target].values

In [65]:

def get_list_contains_ohe(keywords, cols_list):
    ouput = []
    for keyword in keywords:
        ouput.append( [1 if keyword in i else 0 for i in cols_list] )
    return ouput

# Tiếp cận theo hướng recommend - cell -> chemical | cell/gene: user, chemial: item
n_components = 350

u_fts_num = X_train.shape[1]#num_fts
i_fts_num = num_labels

initializer = 'he_normal'

#User embedding
input_u = Input(shape = (u_fts_num,) , name ='input_u1' )
layer_u = BatchNormalization( ) (input_u)
layer_u = Dropout(0.25 ) (layer_u)
layer_u = WeightNormalization(Dense(1024, activation="selu", kernel_initializer= initializer, kernel_regularizer= tf.keras.regularizers.l2(0.0001) )) (layer_u)
layer_u = BatchNormalization( ) (layer_u)
layer_u = Dropout(0.25 ) (layer_u)
layer_u = WeightNormalization(Dense(1024, activation="selu", kernel_initializer=initializer, kernel_regularizer= tf.keras.regularizers.l2(0.0001) )) (layer_u)
layer_u = BatchNormalization( ) (layer_u)
layer_u = Dropout(0.25 ) (layer_u)
layer_u = WeightNormalization(Dense(n_components, activation = 'selu', kernel_initializer= initializer, kernel_regularizer= tf.keras.regularizers.l2(0.0001) )) (layer_u)
layer_u = BatchNormalization() (layer_u)

#Item embedding
  # Addition information for item_info
list_chem_gr = ['_inhibitor', '_agonist', '_agent', '_antagonist', '_blocker', '_activator']
chemical_category = tf.transpose( tf.constant( get_list_contains_ohe( list_chem_gr, cols_target  ) ))
  # Full item fts: addition + onehot
item_ft = tf.concat(
    [chemical_category ,
     tf.eye(i_fts_num, dtype = tf.int32) # Create tensor 0-1 coresponse with chemical labels
    ], axis = 1
)
layer_i = WeightNormalization(Dense(n_components, activation = 'selu', kernel_initializer= initializer, kernel_regularizer= tf.keras.regularizers.l2(0.0001) )) (item_ft)
layer_i = BatchNormalization() (layer_i)

# Dot product user - item
def dot_2layer(x):
    return K.dot( x[0], K.transpose(x[1]))
dot_ui = Lambda( dot_2layer, name = 'lambda_dot' ) ([layer_u,layer_i])
dot_ui= BatchNormalization() (dot_ui)
dot_ui= WeightNormalization(Dense(512, activation="selu" , kernel_initializer= initializer, kernel_regularizer= tf.keras.regularizers.l2(0.0001))) (dot_ui)
dot_ui= BatchNormalization() (dot_ui)
dot_ui= WeightNormalization(Dense(256, activation="selu", kernel_initializer= initializer, kernel_regularizer= tf.keras.regularizers.l2(0.0001) )) (dot_ui)
dot_ui= BatchNormalization() (dot_ui)
dot_ui = Dense(i_fts_num, activation = 'sigmoid' )(dot_ui)

# Compile model
model = Model(inputs=[input_u, ], outputs= [dot_ui])

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

bce = tf.keras.losses.BinaryCrossentropy()
model.compile(loss= BinaryCrossentropy(label_smoothing=0.001), optimizer=opt 
              , metrics= [bce])
# print( model.summary() )

# tf.keras.utils.plot_model(model,show_shapes=True)

In [66]:
# help(tfa.optimizers.CyclicalLearningRate)

In [67]:
reduce_lr = ReduceLROnPlateau(monitor='val_binary_crossentropy', factor=0.1, patience=5, mode='min', min_lr=1E-5, verbose= 0)
early_stopping = EarlyStopping(monitor='val_binary_crossentropy', min_delta=1E-5, patience=15, mode='min',restore_best_weights=True, verbose= 0)
    
model.fit(
        X_train, y_train, validation_split = 0.25, 
        callbacks=[reduce_lr, early_stopping], epochs=150, verbose =1,
        batch_size=32 )

Epoch 1/150
515/515 [==============================] - 6s 12ms/step - loss: 1.7539 - binary_crossentropy: 0.6148 - val_loss: 1.3815 - val_binary_crossentropy: 0.3970
Epoch 2/150
515/515 [==============================] - 6s 11ms/step - loss: 1.0631 - binary_crossentropy: 0.1998 - val_loss: 0.8560 - val_binary_crossentropy: 0.0880
Epoch 3/150
515/515 [==============================] - 6s 11ms/step - loss: 0.7749 - binary_crossentropy: 0.0566 - val_loss: 0.7210 - val_binary_crossentropy: 0.0371
Epoch 4/150
515/515 [==============================] - 6s 12ms/step - loss: 0.6996 - binary_crossentropy: 0.0311 - val_loss: 0.6833 - val_binary_crossentropy: 0.0254
Epoch 5/150
515/515 [==============================] - 6s 11ms/step - loss: 0.6763 - binary_crossentropy: 0.0236 - val_loss: 0.6723 - val_binary_crossentropy: 0.0236
Epoch 6/150
515/515 [==============================] - 6s 11ms/step - loss: 0.6672 - binary_crossentropy: 0.0206 - val_loss: 0.6658 - val_binary_crossentropy: 0.0210
Epoc

In [14]:
prediction = model.predict(X_pred)
df_preds_non_ctl =  pd.DataFrame(prediction, columns= cols_target, index = to_pred.index)

# concat with all to pred values
df_preds = pd.concat([ full_pred[cols_id], df_preds_non_ctl], axis = 1).fillna(0)

df_preds.iloc[:,[34,82]] = 0
# to csv
df_preds.to_csv("submission.csv", index = None)

In [15]:
df_preds

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.005583,0.001314,0.001375,0.012943,0.038262,0.005347,0.004701,0.004893,0.000287,0.011884,0.019809,0.000697,0.000481,0.000977,0.001719,0.000897,0.004366,0.008201,0.008962,0.002465,0.002560,0.006375,0.000394,0.002246,0.000680,0.000542,0.001457,0.001167,0.010180,0.003589,0.002711,0.003168,0.005202,0,0.000672,0.000414,0.002110,0.000349,0.000867,...,0.003634,0.001439,0.008521,0.000562,0.000717,0.005229,0.000731,0.001123,0.001188,0.001752,0.013692,0.015655,0.003518,0.006351,0.002917,0.001300,0.029287,0.002469,0.000672,0.000559,0.000298,0.003552,0.000776,0.001336,0.001713,0.004114,0.001161,0.001678,0.002045,0.001615,0.000935,0.000554,0.003373,0.001401,0.000506,0.000892,0.001066,0.001678,0.001374,0.001015
1,id_001897cda,0.000153,0.000285,0.000130,0.000326,0.000113,0.000206,0.000898,0.006111,0.026226,0.003814,0.002276,0.001464,0.000102,0.004161,0.000324,0.000481,0.000167,0.001675,0.000608,0.000335,0.001512,0.000610,0.000814,0.000488,0.000559,0.001593,0.000165,0.000060,0.000229,0.000647,0.000571,0.000906,0.000417,0,0.000387,0.000109,0.003098,0.001348,0.002504,...,0.000191,0.000461,0.000070,0.000590,0.001441,0.002988,0.000317,0.012005,0.000143,0.004533,0.003265,0.002142,0.000332,0.000335,0.001604,0.000362,0.001501,0.000125,0.037203,0.000179,0.011030,0.001294,0.003802,0.000623,0.000398,0.000197,0.002493,0.000642,0.005461,0.001560,0.000421,0.000457,0.001082,0.000038,0.009953,0.000258,0.007813,0.000701,0.002730,0.002760
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.000339,0.0015